In [1]:
# https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6
from utils.folium_utils import get_copenhagen_grid, get_geojson_grid, get_coords
from ipywidgets import interact, interact_manual
from data_api import *

stations = select_stations_join_departures()
bus_ids = get_bus_ids()
routes = get_routes_geojson(bus_ids)
grid_geojson = get_grid_geojson()

In [2]:
merge_geojson(routes, grid_geojson)

In [3]:
def get_grid(start_hour):
    end_hour = start_hour + 1
    if start_hour == 23:
        end_hour = 0
    f_start_hour = '{}:00'.format(str(start_hour))
    f_end_hour = '{}:00'.format(str(end_hour))
    filtered_stations = filter_by_hour(stations, f_start_hour, f_end_hour)
    return get_copenhagen_grid(filtered_stations)
    # return stations_2['time'] # use for checking if filter works

# maybe possible to make it faster
m = interact(get_grid, start_hour=widgets.IntSlider(min=0, max=23, step=1, value=0))

interactive(children=(IntSlider(value=0, description='start_hour', max=23), Output()), _dom_classes=('widget-i…

In [4]:
coords = stations.iloc[0].coordinates_here.split(',')
m = folium.Map([coords[0], coords[1]], zoom_start=10)
stations[['latitude', 'longitude']] = stations.apply(get_coords, axis=1, result_type="expand")

In [5]:
frequencies = stations.name_overpass.value_counts()

In [6]:
rows = []
for station in frequencies.keys():
    first = stations.loc[stations['name_overpass'] == station].iloc[0]
    rows.append({'station_name': station,
                 'longitude': first.longitude,
                 'latitude': first.latitude,
                 'bus_count': frequencies.get(station)})

final = pd.DataFrame(rows)
final['longitude'] = final['longitude'].astype(float)
final['latitude'] = final['latitude'].astype(float)
top_right = [final['latitude'].max(), final['longitude'].max()]
bottom_left = [final['latitude'].min(), final['longitude'].min()]

In [7]:
# Generating a grid heatmap (utils/folium_utils.py)
grid = get_geojson_grid(top_right, bottom_left, n=10)

In [8]:
# Frequencies by latitude and longitude
counts = []
for box in grid:
    upper_right = box["properties"]["upper_right"]
    lower_left = box["properties"]["lower_left"]
    items = final.loc[(final['latitude'] <= upper_right[1]) &
                      (final['longitude'] <= upper_right[0]) &
                      (final['longitude'] >= lower_left[0]) &
                      (final['latitude'] >= lower_left[1])]
    counts.append(len(items))
max_count = max(counts)

In [9]:
# Add GeoJson to map
for i, geo_json in enumerate(grid):
    color = plt.cm.Reds(counts[i] / max_count)
    color = mpl.colors.to_hex(color)
    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, 
                        color=color: {
                            'fillColor': color,
                            'color': "black",
                            'weight': 2,
                            'dashArray': '5, 5',
                            'fillOpacity': 0.55,
                        })

    m.add_child(gj)

In [10]:
m